In [1]:
# ! pip install py-hanspell

In [6]:
#전처리에 필요한 라이브러리
import os
import re
import sys
sys.path.append("C:\\Users\\hun98\\data_mining\\result\\henna")
sys.path.append("C:\\Users\\hun98\\data_mining\\result\\lescape")
sys.path.append("C:\\Users\\hun98\\data_mining\\result\\novotel")
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from hanspell import spell_checker
from tqdm import tqdm

# creat henna hotel dataset

In [194]:
data_path = os.path.join(os.getcwd(),"result\\henna")

In [195]:
file_list = os.listdir(data_path)

In [196]:
file_list

['booking_dot_henna.csv',
 'expedia_henna.csv',
 'goodchoice_henna.csv',
 'trip_dot_com_henna.csv',
 'yanolja_henna.csv']

In [197]:
dataframe  = pd.read_csv(os.path.join(data_path,file_list[0]),encoding='cp949')

In [198]:
for data in file_list[1:]:
    try:
        new_data = pd.read_csv(os.path.join(data_path,data),encoding='cp949')
        dataframe = pd.concat([dataframe,new_data])
    except:
        new_data = pd.read_csv(os.path.join(data_path,data))
        dataframe = pd.concat([dataframe,new_data])

In [199]:
dataframe=dataframe.drop(['index','Unnamed: 0'],axis=1)

In [200]:
dataframe.columns

Index(['user_name', 'date', 'point', 'txt'], dtype='object')

In [201]:
del dataframe['user_name']

In [202]:
del dataframe['date']

In [203]:
del dataframe['point']

In [204]:
dataframe=dataframe.astype({'txt':'str'})

In [205]:
dataframe.reset_index(drop=False,inplace=True)

In [206]:
del dataframe['index']

In [207]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 575 entries, 0 to 574
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   txt     575 non-null    object
dtypes: object(1)
memory usage: 4.6+ KB


# henna hotel text preprocessing

In [208]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U000E007F"  # 이모티콘
        u"\U0001F300-\U0001F5FF"  # 픽토그램
        u"\U0001F680-\U0001F6FF"  # 다른 그림들
        u"\U0001F1E0-\U0001F1FF"  # 플래그 (iOS)♥
                           "]+", flags=re.UNICODE)

In [209]:
def clean_text(text):
    text = re.sub('[①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮@$☆◇★○●◎◇◆□■△▲▽▼→←←↑↓↔〓◁◀▷▶♤♠♡♥♧♣⊙◈▣◐◑▒▤▥▨▧▦▩♨☏☎☜☞¶†‡↕↗↙↖↘]', '', text)
    text = re.sub(r"http[s]?://(?:[\t\n\r\f\v]|[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", '', text) # http로 시작되는 url
    text = re.sub(r"[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{2,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)", '', text) # http로 시작되지 않는 url
    text = re.sub(emoji_pattern, '',text)
    text = text.replace("❤️","")
    text = text.replace("☺️","")
    text = re.sub('\([^])]*\)', '', text) # 괄호 제거
    text = re.sub('<[^]>]*>', '', text)
    text = re.sub('\[[^]]*\]', '', text)
    text = text.replace("\n", "")
    text = text.replace("\t", "")
    text = text.replace("\r", "")
    text = text.replace("      "," ")
    text = text.replace("    "," ")
    text = text.replace("   "," ")
    text = text.replace("  "," ")
    text = text.replace("nan","")
    return text

In [210]:
# 텍스트 전처리 이전
for i in range(0,100):
    print(dataframe['txt'][i])


금연호텔인데 이상하게 계속 샤워실에서 담배냄새가 너무 나서 목욕이 불가능했음
겨울 치고 꽤나 더운 날이었는데 중앙난방이어서 에어컨 사용이 불가능했음

비대면 로봇 숙소여서 진귀한 경험 가능
남산 보이는 숙소인데 가성비도 좋음

2022년 11월에 숙박함



매우 만족합니다..
블만없습니다

넘~~~좋아요 그리고 청결하고 직원분들이 매우 친절해요 제가 한국에 오면 전 여기 다시 올거에요 최고에요 넠 편안하고 안전하고 좋았어요 위치도 최고 시설도 최고 !!!
별 다섯게도 아깝지 않은 호텔 ?? 최고!!! ☆☆☆☆☆☆

2022년 11월에 숙박함



없어요 완전 짱

처음으로 1층에 묵어봤어요 예전엔 전망도 없고 로비근처라 시끄럽지 않을까 걱정 해서 기피했는데 아니었어요 방도 넓고 쾌적한데다가 전혀 시끄럽지 않고 지하에 세탁실도 훨씬 오가기 편했어요 편의점이나 잠깐 앞으로 나가기도 훨씬 편했구요

2022년 10월에 숙박함



없습니다 너무 좋았어요 서울오면 무조건 다시 묵어야겠아요

1층 로비에서 식권 구매하면 아침을 먹을 수 있었는데, 이른 시간이라 객실로 포장해갔습니다. 5900원 가격대비 너무 너무 맛있고 양도 충분했어요 ! 음식을 바로 조리하시는지 따뜻하고 신선했습니다!

2022년 9월에 숙박함



샤워하는 도중에 욕조의 물이 제대로 내려가지를 않아 발목까지 물에 잠긴 상태로 샤워를 했습니다. 816호 욕조 하수 관리 꼭 필요해봅니다.

2022년 10월에 숙박함



아쉬운 점은 없으나, 아무래도 비지니스 호텔이니 공간이 부족했습니다.

위치가 좋고 가성비가 좋았습니다
명동에서 묵는 다면 다시 갈거에요
안전했습니다
깨끗했습니다.

2022년 8월에 숙박함





매우 깨끗했어요. 위치도 너무좋았구요. 직원도 매우 친절했습니다.

2022년 9월에 숙박함



현재까지는 없습니다.

비대면 체크인/아웃
최신 호텔이라 깨끗하고 청결함의 끝판왕 객실 상태

2022년 8월에 숙박함



없습니다.

화장실과 욕실이 분리되어 있는 구조 너무 좋았어요

In [211]:
dataframe['txt'] = dataframe['txt'].apply(clean_text)

In [212]:
# 텍스트 전처리 이후
for i in range(0,100):
    print(dataframe['txt'][i])

금연호텔인데 이상하게 계속 샤워실에서 담배냄새가 너무 나서 목욕이 불가능했음겨울 치고 꽤나 더운 날이었는데 중앙난방이어서 에어컨 사용이 불가능했음비대면 로봇 숙소여서 진귀한 경험 가능남산 보이는 숙소인데 가성비도 좋음2022년 11월에 숙박함
매우 만족합니다..블만없습니다넘~~~좋아요 그리고 청결하고 직원분들이 매우 친절해요 제가 한국에 오면 전 여기 다시 올거에요 최고에요 넠 편안하고 안전하고 좋았어요 위치도 최고 시설도 최고 !!!별 다섯게도 아깝지 않은 호텔 ?? 최고!!! 2022년 11월에 숙박함
없어요 완전 짱처음으로 1층에 묵어봤어요 예전엔 전망도 없고 로비근처라 시끄럽지 않을까 걱정 해서 기피했는데 아니었어요 방도 넓고 쾌적한데다가 전혀 시끄럽지 않고 지하에 세탁실도 훨씬 오가기 편했어요 편의점이나 잠깐 앞으로 나가기도 훨씬 편했구요2022년 10월에 숙박함
없습니다 너무 좋았어요 서울오면 무조건 다시 묵어야겠아요1층 로비에서 식권 구매하면 아침을 먹을 수 있었는데, 이른 시간이라 객실로 포장해갔습니다. 5900원 가격대비 너무 너무 맛있고 양도 충분했어요 ! 음식을 바로 조리하시는지 따뜻하고 신선했습니다!2022년 9월에 숙박함
샤워하는 도중에 욕조의 물이 제대로 내려가지를 않아 발목까지 물에 잠긴 상태로 샤워를 했습니다. 816호 욕조 하수 관리 꼭 필요해봅니다.2022년 10월에 숙박함
아쉬운 점은 없으나, 아무래도 비지니스 호텔이니 공간이 부족했습니다.위치가 좋고 가성비가 좋았습니다명동에서 묵는 다면 다시 갈거에요안전했습니다깨끗했습니다.2022년 8월에 숙박함
매우 깨끗했어요. 위치도 너무좋았구요. 직원도 매우 친절했습니다.2022년 9월에 숙박함
현재까지는 없습니다.비대면 체크인/아웃최신 호텔이라 깨끗하고 청결함의 끝판왕 객실 상태2022년 8월에 숙박함
없습니다.화장실과 욕실이 분리되어 있는 구조 너무 좋았어요.수건요청 할 때 로봇이 가져다 준 것도 신기한 경험이었구요.곤란한 일이 있으면 프론트 전화로 직원분과 연결이 되는부분도 좋았어

In [213]:
# hanspell의 spell_checker를 활용해 맞춤법 검사를 진행
spell_check = lambda x : spell_checker.check(x).checked

In [214]:
dataframe['txt'][100]

'생각했던거 보다 조금 더 작은 느낌이었지만 가족과 하루동안 자고가기엔 딱 적당했고 깨끗해서 좋았습니다'

In [215]:
spell_check(dataframe['txt'][100])

'생각했던 거 보다 조금 더 작은 느낌이었지만 가족과 하루 동안 자고 가기엔 딱 적당했고 깨끗해서 좋았습니다'

In [216]:
index_cnt = []
for i in tqdm(range(dataframe.shape[0])):
    try :
        dataframe['txt'][i] = spell_check(dataframe['txt'][i])
    except :
        index_cnt.append(i)

100%|████████████████████████████████████████████████████████████████████████████████| 575/575 [00:50<00:00, 11.29it/s]


In [217]:
index_cnt

[386]

In [218]:
dataframe.drop(index=index_cnt, inplace=True)
dataframe.reset_index(drop=True, inplace=True)

In [219]:
# 텍스트 전처리 이후
for i in range(0,100):
    print(dataframe['txt'][i])

금연 호텔인데 이상하게 계속 샤워실에서 담배 냄새가 너무 나서 목욕이 불가능했음 겨울 치고 꽤나 더운 날이었는데 중앙난방이어서 에어컨 사용이 불가능했음 비대면 로봇 숙소여서 진귀한 경험 가능 남산 보이는 숙소인데 구성비도 좋음 2022년 11월에 숙박함
매우 만족합니다..블만없습니다넘~~~좋아요 그리고 청결하고 직원분들이 매우 친절해요 제가 한국에 오면 전 여기 다시 올 거예요 최고예요 넠 편안하고 안전하고 좋았어요 위치도 최고 시설도 최고!!! 별 다섯 개도 아깝지 않은 호텔 ?? 최고!!! 2022년 11월에 숙박함
없어요 완전 짱 처음으로 1층에 묵어봤어요 예전엔 전망도 없고 로비 근처라 시끄럽지 않을까 걱정해서 기피했는데 아니었어요 방도 넓고 쾌적한데다가 전혀 시끄럽지 않고 지하에 세탁실도 훨씬 오가기 편했어요 편의점이나 잠깐 앞으로 나가기도 훨씬 편했고요 2022년 10월에 숙박함
없습니다 너무 좋았어요 서울 오면 무조건 다시 묵어야겠아요 1층 로비에서 식권 구매하면 아침을 먹을 수 있었는데, 이른 시간이라 객실로 포장해갔습니다. 5900원 가격 대비 너무너무 맛있고 양도 충분했어요! 음식을 바로 조리하시는지 따뜻하고 신선했습니다! 2022년 9월에 숙박함
샤워하는 도중에 욕조의 물이 제대로 내려가지를 않아 발목까지 물에 잠긴 상태로 샤워를 했습니다. 816호 욕조 하수 관리 꼭 필요해봅니다. 2022년 10월에 숙박함
아쉬운 점은 없으나, 아무래도 비즈니스 호텔이니 공간이 부족했습니다. 위치가 좋고 구성비가 좋았습니다 명동에서 묵는다면 다시 갈 거예요 안전했습니다 깨끗했습니다. 2022년 8월에 숙박함
매우 깨끗했어요. 위치도 너무 좋았고요. 직원도 매우 친절했습니다. 2022년 9월에 숙박함
현재까지는 없습니다. 비대면 체크인/아웃 최신 호텔이라 깨끗하고 청결함의 끝판왕 객실 상태 2022년 8월에 숙박함
없습니다. 화장실과 욕실이 분리되어 있는 구조 너무 좋았어요. 수건 요청할 때 로봇이 가져다준 것도 신기한 경험이었고요. 곤란한 일이 있으면 프

In [220]:
short_list =[]
for i in range(dataframe.shape[0]):
    if len(dataframe['txt'].iloc[i])<=2 :
        short_list.append(i)

In [221]:
dataframe.drop(index=short_list, inplace=True)
dataframe.reset_index(drop=True, inplace=True)

In [224]:
dataframe.to_csv("C:\\Users\\hun98\\data_mining\\result\\henna\\henna_dataset.csv")

# Creat lescape hotel dataset

In [121]:
data_path = os.path.join(os.getcwd(),"result\\lescape")
file_list = os.listdir(data_path)
file_list

['expedia_lescape.csv',
 'goodchoice_lescape.csv',
 'hotelscombined_lescape.csv',
 'trip_dot_com_lescape.csv',
 'yanolja_lescape.csv']

In [122]:
dataframe  = pd.read_csv(os.path.join(data_path,file_list[0]),encoding='cp949')
for data in file_list[1:]:
    try:
        new_data = pd.read_csv(os.path.join(data_path,data),encoding='cp949')
        dataframe = pd.concat([dataframe,new_data])
    except:
        new_data = pd.read_csv(os.path.join(data_path,data))
        dataframe = pd.concat([dataframe,new_data])

In [124]:
dataframe.reset_index(drop=False,inplace=True)

In [126]:
dataframe = dataframe[['txt']]

,txt
0,NaN
1,NaN
2,"위치, 방 크기 전부 만족입니다"
3,샤워실 뜨거운물 수압이 낮아서 씻기 힘듦 체크인시 대기시간이 20-30분 정도로 매...
4,NaN
...,...
2463,컨디션좋은 호텔
2464,재밌게 놀다갑니다
2465,비싸지만 좋아요
2466,좋습니다!


In [127]:
dataframe.dropna(inplace=True)
dataframe.reset_index(drop=True,inplace=True)

C:\Users\hun98\AppData\Local\Temp\ipykernel_10724\2825686145.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe.dropna(inplace=True)


In [128]:
dataframe

,txt
0,"위치, 방 크기 전부 만족입니다"
1,샤워실 뜨거운물 수압이 낮아서 씻기 힘듦 체크인시 대기시간이 20-30분 정도로 매...
2,첫째날은 모기때문에 잠 설침. 모기는 어디에나 있나봄. 호텔이라도 어쩔수가 없음. ...
3,넓은 방인데도 시설이 청결합니다. 음성인식이 생각보다 편안합니다
4,"부족함 없는 호텔이에요! 룸 크기도 크고, 컨셉이 확실해서 여행온 느낌 가득 느낄 ..."
...,...
2275,컨디션좋은 호텔
2276,재밌게 놀다갑니다
2277,비싸지만 좋아요
2278,좋습니다!


# lescape hotel text preprocessing

In [133]:
# 텍스트 전처리 이전
for i in range(0,100):
    print(dataframe['txt'][i])

위치, 방 크기 전부 만족입니다
샤워실 뜨거운물 수압이 낮아서 씻기 힘듦 체크인시 대기시간이 20-30분 정도로 매우 김 체크아웃 시 엘레베이터 대기시간이 10분 이상이여서 매우 불편함 어메니티가 없음
첫째날은 모기때문에 잠 설침. 모기는 어디에나 있나봄. 호텔이라도 어쩔수가 없음. 한마리 잡았는데 나머지 한마리가 밤새 윙윙거림. 2박중 둘째날은 옆객실에 묵은 여성분들 수다가 새벽까지 들림. 방음이 아주 훌륭하진 않음.
넓은 방인데도 시설이 청결합니다. 음성인식이 생각보다 편안합니다
부족함 없는 호텔이에요! 룸 크기도 크고, 컨셉이 확실해서 여행온 느낌 가득 느낄 수 있습니다.
엘레베이터기 느린거빼고는 만족합니다.
부티크 호텔 느낌은 좋음 객실 컨디션 나쁘지 않음 묵은 먼지 및 오래된 타올 사용 개선 필요해 보입니다.
대체적으로 만족스러웠습니다 유니크한 인테리어가 가장 강점인 호텔이였고 처음 묵었던 방에 티비가 문제였는데 점검해보시고 바로 방도 바꿔줬어요.
분위기도 좋고 교통편도 좋고 청결하고 여러모로 편히 쉴 수 있어서 만족했습니다
유럽의 로컬 호텔에 온듯한 분위기와 명동 근처라는 위치가 좋네요.
호텔 분위기 배딩 상태 등 너무 만족했고 세이보리 라운지 및 조식 퀄리티도 너무 좋아서 만족 또 만족한 호텔입니다 위치도 좋구요!!
서울내 프랑스 느낌을 받을 수 있던 호텔이었습니다. 뭐 인테리어부터 음식까지 다 마음에 들었지만 제일 마음에 들었던 것은 직원분들의 친절함이었습니다. 한분도 빠짐없이 인사 해주시고 친절히 응대 해주셔서 대접 받는다는 느낌이 들었습니다. 시설은 너무 만족 하였고.. 다만 아쉬운게 있다면 피트니스 시설을 예약하지 못해서 사용 하지 못한게 아쉬웠습니다. 나머지는 다 만족이요 !! 엘베에서 나오는 프랑스어도 정겨웠어요 ㅎ
인테리어가 매우 독특하고 인상적임
호텔이 깨끗하고 분위기가 좋네요
독톡한 분위기와 디자인의 호텔이라 사진 많이 찍고 싶은 분께 추천 직원도 친절하고 룸업그레이드도 해줘서 감사했지만 룸 청소상태는 모텔보다 못한 수준이라 매우 

In [130]:
dataframe['txt'] = dataframe['txt'].apply(clean_text)

C:\Users\hun98\AppData\Local\Temp\ipykernel_10724\3198255081.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['txt'] = dataframe['txt'].apply(clean_text)


In [131]:
# 텍스트 전처리 이후
for i in range(0,100):
    print(dataframe['txt'][i])

위치, 방 크기 전부 만족입니다
샤워실 뜨거운물 수압이 낮아서 씻기 힘듦 체크인시 대기시간이 20-30분 정도로 매우 김 체크아웃 시 엘레베이터 대기시간이 10분 이상이여서 매우 불편함 어메니티가 없음
첫째날은 모기때문에 잠 설침. 모기는 어디에나 있나봄. 호텔이라도 어쩔수가 없음. 한마리 잡았는데 나머지 한마리가 밤새 윙윙거림. 2박중 둘째날은 옆객실에 묵은 여성분들 수다가 새벽까지 들림. 방음이 아주 훌륭하진 않음.
넓은 방인데도 시설이 청결합니다. 음성인식이 생각보다 편안합니다
부족함 없는 호텔이에요! 룸 크기도 크고, 컨셉이 확실해서 여행온 느낌 가득 느낄 수 있습니다.
엘레베이터기 느린거빼고는 만족합니다.
부티크 호텔 느낌은 좋음 객실 컨디션 나쁘지 않음 묵은 먼지 및 오래된 타올 사용 개선 필요해 보입니다.
대체적으로 만족스러웠습니다 유니크한 인테리어가 가장 강점인 호텔이였고 처음 묵었던 방에 티비가 문제였는데 점검해보시고 바로 방도 바꿔줬어요.
분위기도 좋고 교통편도 좋고 청결하고 여러모로 편히 쉴 수 있어서 만족했습니다
유럽의 로컬 호텔에 온듯한 분위기와 명동 근처라는 위치가 좋네요.
호텔 분위기 배딩 상태 등 너무 만족했고 세이보리 라운지 및 조식 퀄리티도 너무 좋아서 만족 또 만족한 호텔입니다 위치도 좋구요!!
서울내 프랑스 느낌을 받을 수 있던 호텔이었습니다. 뭐 인테리어부터 음식까지 다 마음에 들었지만 제일 마음에 들었던 것은 직원분들의 친절함이었습니다. 한분도 빠짐없이 인사 해주시고 친절히 응대 해주셔서 대접 받는다는 느낌이 들었습니다. 시설은 너무 만족 하였고.. 다만 아쉬운게 있다면 피트니스 시설을 예약하지 못해서 사용 하지 못한게 아쉬웠습니다. 나머지는 다 만족이요 !! 엘베에서 나오는 프랑스어도 정겨웠어요 ㅎ
인테리어가 매우 독특하고 인상적임
호텔이 깨끗하고 분위기가 좋네요
독톡한 분위기와 디자인의 호텔이라 사진 많이 찍고 싶은 분께 추천 직원도 친절하고 룸업그레이드도 해줘서 감사했지만 룸 청소상태는 모텔보다 못한 수준이라 매우 

In [134]:
dataframe['txt'][100]

'이색적인 호텔입니다. 외국인손님을 모시거나 커플끼리 한번씩 가보시면 이색적인 데이트가 될꺼라 생각됩니다.^^'

In [135]:
spell_check(dataframe['txt'][100])

'이색적인 호텔입니다. 외국인 손님을 모시거나 커플끼리 한 번씩 가보시면 이색적인 데이트가 될 거라 생각됩니다.^^'

In [136]:
index_cnt = []
for i in tqdm(range(dataframe.shape[0])):
    try :
        dataframe['txt'][i] = spell_check(dataframe['txt'][i])
    except :
        index_cnt.append(i)

  0%|                                                                                         | 0/2280 [00:00<?, ?it/s]C:\Users\hun98\AppData\Local\Temp\ipykernel_10724\2255845987.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['txt'][i] = spell_check(dataframe['txt'][i])
100%|██████████████████████████████████████████████████████████████████████████████| 2280/2280 [04:38<00:00,  8.19it/s]


In [138]:
index_cnt

[178, 227, 447, 924, 931, 954, 1521, 1701, 1986]

In [139]:
dataframe.drop(index=index_cnt, inplace=True)
dataframe.reset_index(drop=True, inplace=True)

C:\Users\hun98\AppData\Local\Temp\ipykernel_10724\2035518646.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe.drop(index=index_cnt, inplace=True)


In [143]:
short_list =[]
for i in range(dataframe.shape[0]):
    if len(dataframe['txt'].iloc[i])<=2 :
        short_list.append(i)

In [144]:
dataframe.drop(index=short_list, inplace=True)
dataframe.reset_index(drop=True, inplace=True)

C:\Users\hun98\AppData\Local\Temp\ipykernel_10724\3678201650.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe.drop(index=short_list, inplace=True)


In [145]:
dataframe.shape

(1915, 1)

In [146]:
dataframe.to_csv("C:\\Users\\hun98\\data_mining\\result\\lescape\\lescape_dataset.csv")

# Creat novotel hotel dataset

In [150]:
data_path = os.path.join(os.getcwd(),"result\\novotel")
file_list = os.listdir(data_path)
file_list

['expedia_novotel.csv',
 'goodchoice_novotel.csv',
 'hotelscombined_novotel.csv',
 'trip_dot_com_novotel.csv',
 'yanolja_novotel.csv']

In [151]:
dataframe_novotel  = pd.read_csv(os.path.join(data_path,file_list[0]),encoding='cp949')
for data in file_list[1:]:
    try:
        new_data = pd.read_csv(os.path.join(data_path,data),encoding='cp949')
        dataframe_novotel = pd.concat([dataframe_novotel,new_data])
    except:
        new_data = pd.read_csv(os.path.join(data_path,data))
        dataframe_novotel = pd.concat([dataframe_novotel,new_data])

In [153]:
dataframe_novotel.reset_index(drop=False,inplace=True)

In [154]:
dataframe_novotel = dataframe_novotel[['txt']]

In [156]:
dataframe_novotel.dropna(inplace=True)
dataframe_novotel.reset_index(drop=True,inplace=True)

In [157]:
dataframe_novotel

,txt
0,직원분들 친절. 건물 전체 및 룸 컨디션 좋았음.
1,시내가 좋습니다 방이크고 깨끗해요
2,와이파이가 느리고 정보가 없어서 불편했습니다
3,호텔이 깔끔하고 좋았습니다. 2박 연박했는데 무난했어요
4,룸에서 냄새가 나서 힘들었어요. 공기청정기라도 비치하세요.ㅃ
...,...
4392,깨끗해서 좋았어요 다만주차장이ㅋ음..
4393,레지던스로 다녀왔어요 너무 넓고 좋았음 루프탑 수영장도 성수기 지나서 횟수제한 없이...
4394,굿굿구ㅛ굿
4395,"기가지니 새롭고 티비도 넓고 침대도 넓습니다. 교통편이야 2,4,5호선 동대문 역사..."


# novotel hotel text preprocessing

In [158]:
dataframe_novotel['txt'] = dataframe_novotel['txt'].apply(clean_text)

In [159]:
index_cnt = []
for i in tqdm(range(dataframe_novotel.shape[0])):
    try :
        dataframe_novotel['txt'][i] = spell_check(dataframe_novotel['txt'][i])
    except :
        index_cnt.append(i)

100%|██████████████████████████████████████████████████████████████████████████████| 4397/4397 [07:12<00:00, 10.17it/s]


In [161]:
dataframe_novotel.drop(index=index_cnt, inplace=True)
dataframe_novotel.reset_index(drop=True, inplace=True)

In [166]:
short_list =[]
for i in range(dataframe_novotel.shape[0]):
    if len(dataframe_novotel['txt'].iloc[i])<=2 :
        short_list.append(i)

In [167]:
dataframe_novotel.drop(index=short_list, inplace=True)
dataframe_novotel.reset_index(drop=True, inplace=True)

In [168]:
dataframe_novotel.shape

(3915, 1)

In [165]:
dataframe_novotel.to_csv("C:\\Users\\hun98\\data_mining\\result\\novotel\\novotel_dataset.csv")